In [86]:
import pandas as pd
import gseapy as gp
import networkx as nx
import math

from itertools import chain, repeat

1.	Realizar a análise de enriquecimento usando o método GSEA

In [87]:
PATH_EXTERNAL_DATA = '../data/external/'
PATH_INTERIM_DATA = '../data/interim/'
PATH_PROCESSED_DATA = '../data/processed/'
PATH_RAW_DATA = '../data/raw/'

In [93]:
files = [#['ZIKV_POL_vs_CTRL_24hpi', 'ZIKV_UGD_vs_CTRL_24hpi', 'ZIKV_all_genes_24hpi'],
         #['ZIKV_POL_vs_CTRL_48hpi', 'ZIKV_UGD_vs_CTRL_48hpi', 'ZIKV_all_genes_48hpi'],
         ['ZIKV_POL_vs_CTRL_72hpi', 'ZIKV_UGD_vs_CTRL_72hpi', 'ZIKV_all_genes_72hpi']]

for flist in files:
    ZIKV_POL_vs_CTRL = pd.read_csv(PATH_RAW_DATA + flist[0] + '.tsv', sep='\t')
    ZIKV_UGD_vs_CTRL = pd.read_csv(PATH_RAW_DATA + flist[1] + '.tsv', sep='\t')
    ZIKV_all_genes = pd.read_csv(PATH_RAW_DATA + flist[2] + '.tsv', sep='\t')


In [94]:
ZIKV_POL_vs_CTRL.rename(columns={"log2(fold change)": "log2FoldChange"}, inplace=True)
ZIKV_UGD_vs_CTRL.rename(columns={"log2(fold change)": "log2FoldChange"}, inplace=True)
ZIKV_all_genes.rename(columns={"GENE_SYMBOL": "Gene_name"}, inplace=True)

ZIKV_POL_vs_CTRL_genes = pd.merge(ZIKV_POL_vs_CTRL, ZIKV_all_genes, on=["ID"])
ZIKV_UGD_vs_CTRL_genes = pd.merge(ZIKV_UGD_vs_CTRL, ZIKV_all_genes, on=["ID"])

In [95]:
ZIKV_POL_vs_CTRL_genes = ZIKV_POL_vs_CTRL_genes[['Gene_name', 'log2FoldChange']]
ZIKV_POL_vs_CTRL_genes = ZIKV_POL_vs_CTRL_genes[~ZIKV_POL_vs_CTRL_genes['Gene_name'].isna()]

ZIKV_UGD_vs_CTRL_genes = ZIKV_UGD_vs_CTRL_genes[['Gene_name', 'log2FoldChange']]
ZIKV_UGD_vs_CTRL_genes = ZIKV_UGD_vs_CTRL_genes[~ZIKV_UGD_vs_CTRL_genes['Gene_name'].isna()]

In [ ]:
down = ZIKV_POL_vs_CTRL_72hpi_genes
down

In [ ]:
# Podemos ver os "gene sets" que estão incluídos no gseapy. Alguns deles não estão atualizados
# Portanto, nesta tarefa vamos usar um arquivo gmt baixado do MSigDB (https://www.gsea-msigdb.org/gsea/msigdb/)
#gp.get_library_name()[:10]

In [56]:

down

,Gene_name,log2FoldChange
0,XLOC_l2_015098,1.352
1,LOC100507165,1.600
2,FLJ35024,1.192
3,XLOC_007868,0.576
4,LOC388796,0.713
...,...,...
871,LOC338653,0.844
872,LOC338620,-0.490
873,DKFZp451B082,1.873
874,JAK2,0.503


In [ ]:
# Gene Set Enrichment Analysis

def proceed_enrichment(down):

    # Executar a análise de enriquecimento
    # O Enrichr usa o Fisher exact test
    #del enrichr_down
    enr = gp.enrichr(gene_list=down.Gene_name, gene_sets=[PATH_EXTERNAL_DATA + 'gene_sets_GOBP.gmt'], outdir=None) 

    #enr = gp.enrichr(gene_list=down.Gene_name,gene_sets=['Reactome_2022', "h.all.v2023.2.Hs.symbols.gmt"], outdir=None) # Para usar a versão do mesmo arquivo gmt do gseapy.get_library_name()
    #resultado da análise
    enrichr_down = enr.results
    enrichr_down.head()
    enrichr_down.shape

    # somente os resultados estatisticamente significativos
    enrichr_down = enrichr_down[enrichr_down["Adjusted P-value"]<0.05]

    enrichr_down.reset_index(drop=True, inplace=True)
    enrichr_down
    enrichr_down.shape

    type(enrichr_down)

    pd.options.mode.chained_assignment = None # disable the warning
    enrichr_down.replace('GOBP_', '', regex=True, inplace=True)
    enrichr_down.replace('_', ' ', regex=True, inplace=True)
    enrichr_down

    # A coluna Genes indica os genes associados com as vias
    # Vamos passar essa informação para um DataFrame com duas colunas (uma via ==> um gene)
    pathways = {}

    for ind in enrichr_down.index:
        
        path = enrichr_down.iloc[ind, 1]
        genes = enrichr_down.loc[ind, "Genes"]

        genes = genes.strip().split(";")

        pathways[path] = genes

        #if len(genes) < 5:
            
            #break
        #else:
            #pathways[path] = genes

        df = pd.DataFrame.from_records(
            chain.from_iterable(zip(repeat(k), v) for k, v in pathways.items()),
            columns=["geneset", "genesymbol"],
        )    

    df.head()
    df.shape

2. rede das vias enriquecidas

In [69]:
G = nx.Graph() # Crear um grafo non direcionado 
G.add_nodes_from(df['geneset']) # adicionar nós
G.add_nodes_from(df['genesymbol'])
edges = [(row['geneset'], row['genesymbol']) for index, row in df.iterrows()] # Definir as arestas
G.add_edges_from(edges) # adicionar as arestas
# Exportando a rede para melhor visualização no Cytoscape
nx.write_gml(G, PATH_INTERIM_DATA + 'ZIKV_POL_vs_CTRL_72hpi_genes' + '.gml')

#import matplotlib.pyplot as plt
## definir o layout
#pos = nx.spring_layout(G)
#nx.draw(G, pos, with_labels=True, node_size=300, node_color='skyblue', font_size=10, font_color='black')
## Mostrar o grafo
#plt.show()